In [70]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup


def parse_years(input_years):
    """Обрабатывает входные параметры годов, поддерживая конкретные годы, списки и диапазоны."""
    if isinstance(input_years, list):
        return [str(year) for year in input_years]
    elif isinstance(input_years, str) and '-' in input_years:
        start_year, end_year = map(int, input_years.split('-'))
        return [str(year) for year in range(start_year, end_year + 1)]  # Включительно конечный год
    elif isinstance(input_years, int):
        return [str(input_years)]  # Преобразование одиночного года в список строк
    return None



def get_seasons_and_meetings(years=None):
    # Fetch initial data for available seasons and meetings from the website
    url = "https://www.gt-world-challenge-europe.com/results?filter_season_id=0&filter_meeting_id=0&filter_race_id="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract available seasons
    seasons_options = soup.select('#filter_season_id option')
    seasons = {option.text.strip(): option['value'] for option in seasons_options if option['value']}

    # Filter seasons by specified years, if provided
    if years:
        seasons = {year: seasons[year] for year in years if year in seasons}
    
    # Extract available meetings
    meetings_options = soup.select('#filter_meeting_id option')
    meetings = {option.text.strip(): option['value'] for option in meetings_options if option['value']}
    
    return seasons, meetings


def get_races(input_years=None):
    # Get race data for specified years or all years if none specified
    base_url = "https://www.gt-world-challenge-europe.com/results"
    years = parse_years(input_years)
    seasons, meetings = get_seasons_and_meetings(years)
    results = []

    for season_name, season_id in seasons.items():
        for meeting_name, meeting_id in meetings.items():
            # Fetch race options for each season and meeting combination
            url = f"{base_url}?filter_season_id={season_id}&filter_meeting_id={meeting_id}&filter_race_id="
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            race_options = soup.select('#filter_race_id option')

            races = []
            for option in race_options:
                if option['value']:  # Exclude empty values
                    races.append({"race_id": option['value'], "name": option.text.strip()})
            
            # Append season, meeting, and races information to the results
            results.append({
                "season": season_name,
                "season_id": season_id,
                "meeting": meeting_name,
                "meeting_id": meeting_id,
                "races": races
            })

    return results


def get_race_results(season, meeting, race_name, race_data):
    # Получение ID сезона, встречи и заезда
    season_id, meeting_id, race_id = None, None, None
    for data in race_data:
        if data['season'] == season and data['meeting'] == meeting:
            season_id = data['season_id']
            meeting_id = data['meeting_id']
            for race in data['races']:
                if race['name'] == race_name:
                    race_id = race['race_id']
                    break
            break
    
    if not season_id or not meeting_id or not race_id:
        raise ValueError(f"Cannot find data for season: {season}, meeting: {meeting}, race: {race_name}")

    # Формирование URL и запрос к сайту
    final_url = f"https://www.gt-world-challenge-europe.com/results?filter_season_id={season_id}&filter_meeting_id={meeting_id}&filter_race_id={race_id}"
    response = requests.get(final_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Проверка наличия таблицы на странице
    table = soup.find('table', {'class': 'table'})
    if table:
        headers = [header.text.strip() for header in table.find('thead').find_all('th')]
        data_rows = [[col.text.strip() for col in row.find_all('td')] for row in table.find('tbody').find_all('tr')]
        return pd.DataFrame(data_rows, columns=headers)
    else:
        print(f"No data table found for race: {race_name} in season: {season}, meeting: {meeting}. Skipping this race.")
        return pd.DataFrame()  # Возвращаем пустой DataFrame для отсутствующих данных






def load_race_data(input_years=None):
    # Load race data, optionally filtered by years
    years = parse_years(input_years)  # Use parse_years to handle the input
    all_data = get_races(years)  # Fetch data using existing filtering logic
    return all_data


def all_info(race_data):
    # Preparing an empty list to store information about each race
    summary_data = []

    # We go through all the downloaded data to collect information
    for data in race_data:
        season = data['season']
        meeting = data['meeting']
        for race in data['races']:
            race_name = race['name']
            # Adding race information to the list
            summary_data.append({
                'Season': season,
                'Meeting': meeting,
                'Race Name': race_name
            })

    # Creating a DataFrame from the collected information
    summary_df = pd.DataFrame(summary_data)

    return summary_df



def save_race_data_to_csv(years="2023", meetings=None, race_type=None, race_data=None, save_path='.', create_folders=False):
    if not race_data:
        print("Нет данных для обработки.")
        return

    if isinstance(years, str):
        years = parse_years(years)
    if isinstance(meetings, str):
        meetings = [meetings]
    if race_type and isinstance(race_type, str):
        race_type = [race_type.lower()]

    data_dir = os.path.join(save_path, 'data_csv')
    os.makedirs(data_dir, exist_ok=True)
    
    for data in race_data:
        if years and data['season'] not in years:
            continue
        if meetings and data['meeting'] not in meetings:
            continue
        
        meeting_dir = os.path.join(data_dir, data['season'], data['meeting'].replace(" ", "_"))
        os.makedirs(meeting_dir, exist_ok=True)
        
        for race in data['races']:
            if race_type:
                # Проверка на соответствие указанному типу заезда
                if not any(r.lower() == race['name'].lower() for r in race_type):
                    continue  # Пропускаем заезд, если он не соответствует указанному типу
            
            # Определение пути для сохранения на основе наличия флага create_folders
            folder_path = meeting_dir
            if create_folders:
                if "qualifying" in race['name'].lower():
                    folder_path = os.path.join(meeting_dir, "Qualifications")
                elif "main race" in race['name'].lower():
                    folder_path = os.path.join(meeting_dir, "Races")
                os.makedirs(folder_path, exist_ok=True)
            
            file_path = os.path.join(folder_path, f"{race['name'].replace(' ', '_')}.csv")
            df = get_race_results(data['season'], data['meeting'], race['name'], race_data)
            df.to_csv(file_path, index=False)
            print(f"Сохранено: {file_path}")



In [49]:
my_race_data = load_race_data(2023)

In [50]:
race_summary_df = all_info(my_race_data)

race_summary_df.head(5)

,Season,Meeting,Race Name
0,2023,Monza,Main Race after 2.30 hours
1,2023,Monza,Main Race after 1.30 hour
2,2023,Monza,Main Race after 30 mins
3,2023,Monza,Main Race
4,2023,Monza,Combined Qualifying


In [65]:
results = race_summary_df[race_summary_df['Meeting'] == 'Monza']['Race Name'].reset_index(drop=True)
results.head(15)

0        Main Race after 2.30 hours
1         Main Race after 1.30 hour
2           Main Race after 30 mins
3                         Main Race
4               Combined Qualifying
5                      Qualifying 3
6                      Qualifying 2
7                      Qualifying 1
8                          Pit walk
9     Fanatec Esports GT Pro Series
10                   Pre Qualifying
11                    Free practice
12                      Bronze Test
Name: Race Name, dtype: object

In [69]:
df = get_race_results("2023", "Monza", "Fanatec Esports GT Pro Series", my_race_data)

df.head()

No data table found for race: Fanatec Esports GT Pro Series in season: 2023, meeting: Monza. Skipping this race.


""


In [72]:
save_race_data_to_csv(years="2023", race_data=my_race_data, create_folders=True)

Сохранено: .\data_csv\2023\Monza\Races\Main_Race_after_2.30_hours.csv
Сохранено: .\data_csv\2023\Monza\Races\Main_Race_after_1.30_hour.csv
Сохранено: .\data_csv\2023\Monza\Races\Main_Race_after_30_mins.csv
Сохранено: .\data_csv\2023\Monza\Races\Main_Race.csv
Сохранено: .\data_csv\2023\Monza\Qualifications\Combined_Qualifying.csv
Сохранено: .\data_csv\2023\Monza\Qualifications\Qualifying_3.csv
Сохранено: .\data_csv\2023\Monza\Qualifications\Qualifying_2.csv
Сохранено: .\data_csv\2023\Monza\Qualifications\Qualifying_1.csv
Сохранено: .\data_csv\2023\Monza\Pit_walk.csv
No data table found for race: Fanatec Esports GT Pro Series in season: 2023, meeting: Monza. Skipping this race.
Сохранено: .\data_csv\2023\Monza\Fanatec_Esports_GT_Pro_Series.csv
Сохранено: .\data_csv\2023\Monza\Qualifications\Pre_Qualifying.csv
Сохранено: .\data_csv\2023\Monza\Free_practice.csv
Сохранено: .\data_csv\2023\Monza\Bronze_Test.csv
Сохранено: .\data_csv\2023\Brands_Hatch\Race_2.csv
Сохранено: .\data_csv\2023\Bra